In [ ]:
import time
import os
from dotenv import load_dotenv

from langchain_openai import OpenAIEmbeddings
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

c:\Users\daniel.senna\AppData\Local\anaconda3\envs\noh-faq\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
load_dotenv()
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

index_name = "noh-faq-index-3072"  
index_large = pc.Index(index_name)

In [3]:
#checando se o index no pinecone foi criado
pc.describe_index("noh-faq-index-3072")

{
    "name": "noh-faq-index-3072",
    "dimension": 3072,
    "metric": "cosine",
    "host": "noh-faq-index-3072-0c15b22.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "deletion_protection": "disabled"
}

In [4]:
embeddings_large = OpenAIEmbeddings(model="text-embedding-3-large")
vector_store_large = PineconeVectorStore(index=index_large, embedding=embeddings_large)

In [8]:
#testando uma query
user_question = "preciso pagar pra ter conta na Noh?"
start_time_large = time.time()
results_large = vector_store_large.similarity_search_with_score(
    user_question,
    k=5
    )
end_time_large = time.time()
tempo_large = end_time_large - start_time_large

In [9]:
print(f"Tempo modelo large: {tempo_large:.4f} segundos")

Tempo modelo large: 1.0833 segundos


In [11]:
for i  in results_large:
    print(i[0].page_content)
    print(i[1])

Quanto custa a Noh? A Noh tem uma mensalidade de R$14,00 por casal (R$ 7,00 por pessoa). Esse valor é o que vai nos ajudar a continuar crescendo de forma sustentável, sem a necessidade de vender seus dados, ou exibir anúncios de outros produtos que poluam sua experiência no aplicativo. Queremos que nossa fonte de receita venha do aplicativo e cartão que criamos, que é o símbolo do nosso movimento para acabar com as gambiarras dos casais que tentam adaptar ferramentas individuais para uma vida conjunta.
0.664116
O que acontece se eu não quiser assinar a Noh? Ao criar a conta, damos um prazo para testarem o app e decidirem se ele faz sentido nas finanças compartilhadas de vocês, mas só assinantes poderão usar a Noh. Caso não queiram assinar, vocês poderão desativar suas contas pelo próprio app. Ao desativá-las, não será mais possível fazer pagamentos com a Noh, os cartões (virtual e físico) serão cancelados e vocês não terão mais acesso ao app.
0.657379508
Por que a mensalidade? A Noh na

In [46]:
contexto_rag = ""
for i in results_large:
    titulo = i[0].metadata['titulo_artigo']
    conteudo = i[0].metadata['conteudo_artigo']
    contexto_rag += f"Título do Artigo: {titulo}\nConteúdo: {conteudo}\n\n"

In [47]:
print(contexto_rag)

Título do Artigo: Quanto custa a Noh?
Conteúdo: A Noh tem uma mensalidade de R$14,00 por casal (R$ 7,00 por pessoa). Esse valor é o que vai nos ajudar a continuar crescendo de forma sustentável, sem a necessidade de vender seus dados, ou exibir anúncios de outros produtos que poluam sua experiência no aplicativo. Queremos que nossa fonte de receita venha do aplicativo e cartão que criamos, que é o símbolo do nosso movimento para acabar com as gambiarras dos casais que tentam adaptar ferramentas individuais para uma vida conjunta.

Título do Artigo: O que acontece se eu não quiser assinar a Noh?
Conteúdo: Ao criar a conta, damos um prazo para testarem o app e decidirem se ele faz sentido nas finanças compartilhadas de vocês, mas só assinantes poderão usar a Noh. Caso não queiram assinar, vocês poderão desativar suas contas pelo próprio app. Ao desativá-las, não será mais possível fazer pagamentos com a Noh, os cartões (virtual e físico) serão cancelados e vocês não terão mais acesso ao 

In [18]:
#gpt-4o-mini
from langchain_openai import ChatOpenAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
#from langchain import hub


In [52]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.6,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

In [54]:
system_template = """TAREFA:
            Você é um assistente para pergunta-resposta (Q&A), seu papel é responder dúvidas de visitantes no website da Noh. Noh é um banco digital que oferece conta financeira para apenas para casais.
            Como contexto, serão fornecidos 3 artigos da central de ajuda da Noh obtidos via embeddings (RAG), ou seja, serão fornecidos os 3 artigos mais similares à pergunta do usuário. 
            INSTRUÇÕES:
            Responda EXCLUSIVAMENTE com base nos artigos fornecidos como contexto, NÃO invente informações sobre a conta Noh. Se você achar que os conteúdos dos artigos fornecidos não são suficientes para responder a pergunta do usuário, apenas diga que não é capaz de responder no momento. 
            Responda em língua portuguesa. 
            Responda com o mesmo tom de voz dos artigos fornecidos como contexto.
            Responda de forma concisa e termine com "para mais informações, consulte os artigos abaixo".

            Artigos da Central de Ajuda:\n{contexto}"""

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{user_question}")]
)

In [55]:
prompt = prompt_template.invoke({"contexto": contexto_rag, "user_question": user_question})

In [56]:
response = llm.invoke(prompt)
print(response.content)

Sim, a Noh tem uma mensalidade de R$14,00 por casal (R$7,00 por pessoa). Esse valor é necessário para que o serviço continue a crescer de forma sustentável. Para mais informações, consulte os artigos abaixo.
